## Final Project: Phase 3 - Clustering Pipeline Optimization
Spring 2024  
Group: Michael Massone and Joseph Nelson Farrell   
DS 5230 Unsupervised Machine Learning  
Professor Steven Morin, PhD  
Due: 04/21/2024  
___

In [33]:
## Libraries

# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random


# preprocessing
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

# data
from sklearn.datasets import load_digits

# clustering
from sklearn.cluster import DBSCAN, KMeans

#external indices
from sklearn.metrics.cluster import adjusted_rand_score, contingency_matrix

# runtime and run tracking
import time
from datetime import datetime

# pathing
from pathlib import Path
import os
import sys

import warnings

# Disable runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)



___

### Start Timer

In [34]:
# track runtime
start = time.time()

___

### Define File Paths

In [35]:
# define path
nb_path = Path(os.getcwd())
print(nb_path)
path = str(nb_path.parent)
print(path)

# path to figs folder
figs_path = path + '/figs'

# path to data
data_path= path + '/data'

# path to src folder
src_path = path + '/src'
print(src_path)

# sys path
sys.path.append(src_path)

/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/notebooks
/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final
/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/src


____

### Import Modules

In [36]:
# modules and util
import dimensionality_reduction as dr
import clustering as cl
from cluster_utils import *
from external_indices_utils import *

___

### Parameters

In [37]:
# transformed data csv file name
design_file = '/curated/shuffled_target.csv'

# target vector csv file name
target_file = "/curated/beans_target.csv"

# sampled target dataframe csv loc
target_vector_shuffled_filename = f'/sampled/sampled_target{str(datetime.now()).replace(" ", "")}.csv'

# sampled design dataframe csv loc
design_matrix_shuffled_filename = f'/sampled/sampled_design{str(datetime.now()).replace(" ", "")}.csv'

# all results dataframe csv loc
all_results_file = f'/results/all_results_{str(datetime.now()).replace(" ", "")}.csv'

# filtered results dataframe csv loc
filtered_results_file = f'/results/filtered_results_{str(datetime.now()).replace(" ", "")}.csv'


____

### Load Data

In [38]:
design_file = '/curated/shuffled_target.csv'
target_vector_shuffled = pd.read_csv(data_path + design_file)

target_file = '/curated/shuffled_design.csv'
design_matrix_shuffled = pd.read_csv(data_path + target_file)


In [39]:
design_matrix_shuffled = design_matrix_shuffled.loc[:, :'numerical__ShapeFactor1']

____

### Sample Data

In [40]:
sample = True
if sample:
    # num samples
    n = 1000

    # random seed
    rand_seed = 42

    # sample target and save copy to csv
    target_vector_shuffled = target_vector_shuffled.sample(n=n, random_state=rand_seed)
    target_vector_shuffled.to_csv(data_path + target_vector_shuffled_filename, index=False)
    print("Sampled target dataframe saved at:", data_path + target_vector_shuffled_filename)


     # sample target and save copy to csv
    design_matrix_shuffled = design_matrix_shuffled.sample(n=n, random_state=rand_seed)
    design_matrix_shuffled.to_csv(data_path + design_matrix_shuffled_filename, index=False)
    print("Sampled design dataframe saved at:", data_path + design_matrix_shuffled_filename)


Sampled target dataframe saved at: /Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/data/sampled/sampled_target2024-04-1714:29:13.525291.csv
Sampled design dataframe saved at: /Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/data/sampled/sampled_design2024-04-1714:29:13.525316.csv


___

### Create capX

In [41]:
# drop ID col
design_matrix_shuffled_noID = design_matrix_shuffled.drop('ID', axis=1)

# convert to ndarray
cap_x = design_matrix_shuffled_noID.to_numpy()
cap_x = design_matrix_shuffled
print(f'cap_x shape: {cap_x.shape}')
cap_x.shape

cap_x shape: (1000, 14)


(1000, 14)

___

### Select Hyperparameters for Gridsearch

In [42]:
# select hyperparams
params1=False
params2=False
params3=False
params4=True

if params1:

    min_dist_list = [0.0, 0.01, 0.1, 0.25, 0.5]
    n_neighbors_list = [10, 25, 50, 100, 200]
    metric_list =['euclidean', 'canberra', 'mahalanobis', 'correlation']
    n_components_list = range(2, 4)


if params2:

    min_dist_list = [0.5]
    n_neighbors_list = [100]
    metric_list = ['euclidean']
    n_components_list = [3]


if params3:

    min_dist_list = [0.0, 0.1, 0.5]
    n_neighbors_list = [10, 25, 100, 200]
    metric_list =['euclidean', 'canberra', 'chebyshev', 'correlation']
    n_components_list = range(2, 4)


if params4:

    min_dist_list = [0.0, 0.1, 0.5]
    n_neighbors_list = [10, 50, 100, 200, 500]
    metric_list =['correlation', 'euclidean']
    n_components_list = range(2, 6)




___

### Gridsearch Algorithm

In [43]:
runs = len(n_components_list)*len(min_dist_list)*len(n_neighbors_list)*len(metric_list)
run = 0
df_row_dict_list = []
for n_components in n_components_list:
  for min_dist in min_dist_list:
    for n_neighbors in n_neighbors_list: 
      for metric in metric_list:
        run +=1
        print('*'*100)
        print(f'Run {run} of {runs}')
        results_dict = dr.umap_dim_red(cap_x, n_neighbors, min_dist, metric, n_components) 
        df_row_dict = cl.clustering(results_dict)
        df_row_dict_list.append(df_row_dict)
results_df = pd.DataFrame(df_row_dict_list)


****************************************************************************************************
Run 1 of 160


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  10
min_dist:  0.0
metric:  correlation
n_components:  2
Hopkin's Statistic = 0.1213009703897284
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  6
Validity Index:  0.8399025655250236
****************************************************************************************************
Run 2 of 160


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  10
min_dist:  0.0
metric:  cosine
n_components:  2
Hopkin's Statistic = 0.082702563243827
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  2
Validity Index:  0.9387943607939297
****************************************************************************************************
Run 3 of 160


...

****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  100
min_dist:  0.5
metric:  cosine
n_components:  5
Hopkin's Statistic = 0.08054633368773426
Fail: Kmeans did not successfully cluster.
zero-size array to reduction operation minimum which has no identity
DBSCAN
Number of Clusters:  4
Validity Index:  0.9545860657011821
****************************************************************************************************
Run 159 of 160


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  200
min_dist:  0.5
metric:  correlation
n_components:  5
Hopkin's Statistic = 0.1177249387807747
Test1 Pass: Kmeans successfully clustered.
Number of Clusters:  4
****************************************************************************************************
Run 160 of 160


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  200
min_dist:  0.5
metric:  cosine
n_components:  5
Hopkin's Statistic = 0.06092145541586417
Fail: Kmeans did not successfully cluster.
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
DBSCAN
Number of Clusters:  4
Validity Index:  0.9437468230604724


___

## All Results

In [44]:
results_df.to_csv(data_path + all_results_file, index=False)
print("Filtered results dataframe saved at:", data_path + all_results_file)

# display max rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
results_df

Filtered results dataframe saved at: /Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/data/results/all_results_2024-04-1714:29:13.525335.csv


,algo,n_clusters_found,n_clusters_db_score_is_min,n_clusters_ch_score_is_max,n_clusters_silhouette_score_is_max,silhouette_score,hopkins_statistic,umap_n_neighbors,umap_min_dist,umap_metric,umap_n_components,trustworthiness,eps,dbscan_min_samples,dbscan_metric,validity_index,cluster_labels
0,dbscan,6,NaN,NaN,NaN,NaN,0.121301,10,0.00,correlation,2,0.900346,0.535269,6.0,euclidean,0.839903,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
1,dbscan,2,NaN,NaN,NaN,NaN,0.082703,10,0.00,cosine,2,0.894450,0.178015,6.0,cosine,0.938794,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
2,dbscan,3,NaN,NaN,NaN,NaN,0.128764,25,0.00,correlation,2,0.897259,0.237863,6.0,cosine,0.856663,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,dbscan,3,NaN,NaN,NaN,NaN,0.072102,25,0.00,cosine,2,0.902647,0.159106,6.0,cosine,0.999546,"[0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
4,dbscan,5,NaN,NaN,NaN,NaN,0.120023,50,0.00,correlation,2,0.900903,0.457435,6.0,euclidean,0.828946,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
5,dbscan,5,NaN,NaN,NaN,NaN,0.047629,50,0.00,cosine,2,0.897420,0.330292,6.0,chebyshev,0.969590,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
6,dbscan,5,NaN,NaN,NaN,NaN,0.142524,100,0.00,correlation,2,0.900440,0.407246,5.0,euclidean,0.849916,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
7,dbscan,3,NaN,NaN,NaN,NaN,0.064756,100,0.00,cosine,2,0.895656,0.154497,4.0,cosine,0.973518,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,dbscan,3,NaN,NaN,NaN,NaN,0.118095,200,0.00,correlation,2,0.867513,0.198757,6.0,cosine,0.932400,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
9,dbscan,2,NaN,NaN,NaN,NaN,0.077945,200,0.00,cosine,2,0.889832,0.179695,6.0,cosine,0.986645,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [45]:
# set max rows back to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

____

### Get n Best Results (kmeans or DBSCAN)

In [56]:
n=20

results_df['combined_score'] = results_df['silhouette_score'].fillna(results_df['validity_index'])
results_df = results_df.sort_values(by='combined_score', ascending=False)
filter = results_df.head(n)
filtered_results_df = results_df[results_df.index.isin(filter.index)]
filtered_results_df = filtered_results_df.drop(columns=['combined_score'])

#filtered_results_df[filtered_results_df['n_clusters_found'] == 7]
filtered_results_df

,algo,n_clusters_found,n_clusters_db_score_is_min,n_clusters_ch_score_is_max,n_clusters_silhouette_score_is_max,silhouette_score,hopkins_statistic,umap_n_neighbors,umap_min_dist,umap_metric,umap_n_components,trustworthiness,eps,dbscan_min_samples,dbscan_metric,validity_index,cluster_labels
56,dbscan,3,NaN,NaN,NaN,NaN,0.073866,100,0.01,correlation,3,0.899723,0.252674,6.0,correlation,0.999936,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
68,dbscan,3,NaN,NaN,NaN,NaN,0.106729,200,0.10,correlation,3,0.868299,0.297113,6.0,correlation,0.999858,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
54,dbscan,3,NaN,NaN,NaN,NaN,0.073052,50,0.01,correlation,3,0.898359,0.268203,6.0,correlation,0.999856,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
44,dbscan,3,NaN,NaN,NaN,NaN,0.072765,50,0.00,correlation,3,0.898747,0.249199,6.0,correlation,0.999725,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
42,dbscan,3,NaN,NaN,NaN,NaN,0.062729,25,0.00,correlation,3,0.897117,0.264367,6.0,correlation,0.999717,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
3,dbscan,3,NaN,NaN,NaN,NaN,0.072102,25,0.00,cosine,2,0.902647,0.159106,6.0,cosine,0.999546,"[0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
62,dbscan,3,NaN,NaN,NaN,NaN,0.086519,25,0.10,correlation,3,0.900207,0.326134,6.0,correlation,0.999485,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
64,dbscan,3,NaN,NaN,NaN,NaN,0.097548,50,0.10,correlation,3,0.895912,0.327744,6.0,correlation,0.999429,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
122,dbscan,4,NaN,NaN,NaN,NaN,0.034338,25,0.00,correlation,5,0.899669,0.265335,6.0,correlation,0.999184,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
142,dbscan,4,NaN,NaN,NaN,NaN,0.049971,25,0.10,correlation,5,0.900277,0.324954,6.0,correlation,0.999174,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."


____

### Save Filtered Results to CSV

In [47]:
filtered_results_df.to_csv(data_path + filtered_results_file, index=False)
print("Filtered results dataframe saved at:", data_path + filtered_results_file)

Filtered results dataframe saved at: /Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/data/results/filtered_results_2024-04-1714:29:13.525353.csv


___

## Runtime

In [48]:
finish = time.time()
hours = int((finish - start) // 3600)
minutes = int(((finish - start) % 3600) // 60)
seconds = int((finish - start) % 60)
print(f"Total Run Time(hh:mm.ss): {hours:02d}:{minutes:02d}.{seconds:02d}")

Total Run Time(hh:mm.ss): 00:28.30
